In [2]:
import numpy as np
import pandas as pd
data = pd.read_csv("train.csv")
print(data.shape)
test_data = pd.read_csv("test.csv")
data

(10980, 12)


,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...
10975,569934458364813313,neutral,American,NaN,Cottopanama85,NaN,0,@AmericanAir followback,NaN,2015-02-23 10:58:58 -0800,"ohio,panama",NaN
10976,568564006329434113,positive,United,NaN,PaulBEsteves,NaN,0,@united thanks for the help. Wish the phone re...,NaN,2015-02-19 16:13:17 -0800,Brooklyn,Eastern Time (US & Canada)
10977,569643648910028801,negative,US Airways,NaN,runfixsteve,NaN,0,@usairways the. Worst. Ever. #dca #customerser...,NaN,2015-02-22 15:43:24 -0800,"St. Augustine, Florida",NaN
10978,568864981917110272,negative,US Airways,NaN,CLChicosky,NaN,0,@nrhodes85: look! Another apology. DO NOT FLY ...,NaN,2015-02-20 12:09:15 -0800,NaN,NaN


In [3]:
from nltk.corpus import stopwords
stop = stopwords.stop('english')
import string
punctuations =list(string.punctuation)
print(punctuations)
a = [' ']
stop=stop+punctuations+a
print(stop)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\91913/nltk_data'
    - 'C:\\Users\\91913\\anaconda3\\nltk_data'
    - 'C:\\Users\\91913\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\91913\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\91913\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [6]:
twitter_categories=set(data['airline_sentiment'])
twitter_categories

{'negative', 'neutral', 'positive'}

In [8]:
documents=[]
test_documents=[]
for category in twitter_categories:
    category_rows=(category==data['airline_sentiment'])
    ap = data[category_rows]
    for i in range(len(ap)):
        x_train_current=data[category_rows]
        x_train_current.reset_index(drop=True,inplace=True)
        documents.append((x_train_current['text'][i],category))                  
import random
random.shuffle(documents)

for i in range(len(test_data)):
    test_documents.append(( test_data['text'][i],'negative'))    

In [4]:
new_documents =[(doc.split(' '),category) for doc,category in documents]
print(new_documents)
new_test_documents =[(doc.split(' '),category) for doc,category in test_documents]
print(new_test_documents)

NameError: name 'documents' is not defined

In [3]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
le = WordNetLemmatizer()

In [10]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADJ
    else:
        return wordnet.NOUN
    

In [11]:
def clean_review(words):
    output_words=[]
    for w in words:
        if w.lower() not in stop:
            pos = pos_tag([w])
            clean_words = le.lemmatize(w,pos=get_simple_pos(pos[0][1]))
            output_words.append(clean_words.lower())
    return output_words

In [ ]:
new_documents2 =[(clean_review(words),category)for doc,category in documents]
print(new_documents2)
new_test_documents =[(clean_review(words),category) for doc,category in test_documents]
print(new_test_documents)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
train_category=[category for doc,category in new_documents2]
train_data = [' '.join(doc) for doc,category in new_documents2]
test_data  = [' '.join(doc) for doc,category in new_test_documents2]
# is going through all data and select the best featuresi.e the most frequent features.
count_vec = CountVectorizer(max_features=3000,ngram_range=(1,3))
x_train_matrix=count_vec.fit_transform(train_data)
x_train_matrix.todense()  

NameError: name 'new_documents2' is not defined

In [14]:
count_vec.get_features_names()

NameError: name 'count_vec' is not defined

In [ ]:
x_test_matrix = count_vec.transform(test_data)
x_test_matrix.todense()

In [ ]:
count_vec.get_features_names()

In [ ]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
svm=LogisticRegression()
svm.fit(x_train_matrix,train_category)
y_predict = svm.predict(x_test_matrix)
print(y_predict)
aa = pd.DataFrame(y_predict)
aa.to_csv('twitter.csv',header=False,index=False)